In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) 
import numpy as np
import pickle

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
input_path = 'H:\RediMinds\VCQI'
train = pd.read_csv(input_path+"\VCQI_clean_train_30_day_complications.csv")
test = pd.read_csv(input_path+"\VCQI_clean_test_30_day_complications.csv")

In [3]:
# Create dummy variables
with open (input_path+'\cat_col_30_day_complications', 'rb') as fp:
    cat_col = pickle.load(fp)

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(categories='auto',handle_unknown='ignore')

nn_train = train.copy()
one_hot_encoded_array = encoder.fit_transform(train[cat_col]).toarray()
column_name = encoder.get_feature_names(cat_col)
nn_train_one =  pd.DataFrame(one_hot_encoded_array, columns= column_name)
nn_train = nn_train.merge(nn_train_one, how = 'left', left_index = True, right_index =True) # create dummy variables
nn_train = nn_train.drop(labels = cat_col, axis = 'columns') # drop original variables

In [4]:
nn_train.head()

,AGEATSURGERY,BMI,CLINICALSIZEmm,PREOPHB,PREOPHT,PREOPWBC,PREOPCREAT,PREOPEGFR,NOOFLESIONS,30_day_COMPLICATIONS,GENDER_0,GENDER_1,GENDER_2,MARITALSTATUS_0,MARITALSTATUS_1,MARITALSTATUS_2,MARITALSTATUS_3,MARITALSTATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,EDUCATION_0,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,ECOG_0,ECOG_1,ECOG_2,ECOG_3,ECOG_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARYKIDNEY_0,SOLITARYKIDNEY_1,SOLITARYKIDNEY_2,SOLITARYKIDNEY_3,BILATERALITYOFTUMOR_0,BILATERALITYOFTUMOR_1,BILATERALITYOFTUMOR_2,SIDEOFTUMOR_0,SIDEOFTUMOR_1,SIDEOFTUMOR_2,SIDEOFTUMOR_3,SIDEOFSURGERY_0,SIDEOFSURGERY_1,SIDEOFSURGERY_2,SIDEOFSURGERY_3,FACE_0,FACE_1,FACE_2,TUMORlOCATION_0,TUMORlOCATION_1,TUMORlOCATION_2,TUMORlOCATION_3,TUMORlOCATION_4,TUMORlOCATION_5,POLARLOCATION_0,POLARLOCATION_1,POLARLOCATION_2,RIMLOCATION_0,RIMLOCATION_1,RIMLOCATION_2,RENALSINUS_0,RENALSINUS_1,RENALSINUS_2,EXOPHYTICRATE_0,EXOPHYTICRATE_1,EXOPHYTICRATE_2,EXOPHYTICRATE_3,CLINICALSIZEGROUP_0,CLINICALSIZEGROUP_1,CLINICALSIZEGROUP_2,CLINICALSIZEGROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,CM_4,CM_5,RADIUSmaximaldiameterincm_0,RADIUSmaximaldiameterincm_1,RADIUSmaximaldiameterincm_2,RADIUSmaximaldiameterincm_3,EXOPHYTICENDOPHYTICPROPERTIES_0,EXOPHYTICENDOPHYTICPROPERTIES_1,EXOPHYTICENDOPHYTICPROPERTIES_2,EXOPHYTICENDOPHYTICPROPERTIES_3,EXOPHYTICENDOPHYTICPROPERTIES_4,NEARNESSOFTUMOUR_0,NEARNESSOFTUMOUR_1,NEARNESSOFTUMOUR_2,NEARNESSOFTUMOUR_3,ANTERIORORPOSTERIOR_0,ANTERIORORPOSTERIOR_1,ANTERIORORPOSTERIOR_2,ANTERIORORPOSTERIOR_3,LOCATIONTOPOLARLINE_0,LOCATIONTOPOLARLINE_1,LOCATIONTOPOLARLINE_2,LOCATIONTOPOLARLINE_3,PARTIALNEPHROINDICATION_0,PARTIALNEPHROINDICATION_1,PARTIALNEPHROINDICATION_2,PARTIALNEPHROINDICATION_3,MULTIFOCALITY_0,MULTIFOCALITY_1,MULTIFOCALITY_2
0,57.0,25.84,29.0,14.70,42.40,9400.00,0.86,91.66000,1.00,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,78.0,31.64,24.8,14.90,44.30,6600.00,0.90,81.61000,1.00,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,43.0,24.74,28.0,16.10,46.70,8000.00,1.20,66.08000,1.00,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,72.0,24.60,25.0,13.91,41.56,7205.87,0.83,67.57520,1.05,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,55.0,24.00,19.0,13.91,41.56,7205.87,1.04,55.01636,1.00,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,

In [5]:
# Create dummy variables
nn_test = test.copy()
one_hot_encoded_array = encoder.transform(test[cat_col]).toarray()
column_name = encoder.get_feature_names(cat_col)
nn_test_one =  pd.DataFrame(one_hot_encoded_array, columns= column_name)
nn_test = nn_test.merge(nn_test_one, how = 'left', left_index = True, right_index =True) # create dummy variables
nn_test = nn_test.drop(labels = cat_col, axis = 'columns') # drop original variables

In [6]:
nn_test.head()

,AGEATSURGERY,BMI,CLINICALSIZEmm,PREOPHB,PREOPHT,PREOPWBC,PREOPCREAT,PREOPEGFR,NOOFLESIONS,30_day_COMPLICATIONS,GENDER_0,GENDER_1,GENDER_2,MARITALSTATUS_0,MARITALSTATUS_1,MARITALSTATUS_2,MARITALSTATUS_3,MARITALSTATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,EDUCATION_0,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,ECOG_0,ECOG_1,ECOG_2,ECOG_3,ECOG_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARYKIDNEY_0,SOLITARYKIDNEY_1,SOLITARYKIDNEY_2,SOLITARYKIDNEY_3,BILATERALITYOFTUMOR_0,BILATERALITYOFTUMOR_1,BILATERALITYOFTUMOR_2,SIDEOFTUMOR_0,SIDEOFTUMOR_1,SIDEOFTUMOR_2,SIDEOFTUMOR_3,SIDEOFSURGERY_0,SIDEOFSURGERY_1,SIDEOFSURGERY_2,SIDEOFSURGERY_3,FACE_0,FACE_1,FACE_2,TUMORlOCATION_0,TUMORlOCATION_1,TUMORlOCATION_2,TUMORlOCATION_3,TUMORlOCATION_4,TUMORlOCATION_5,POLARLOCATION_0,POLARLOCATION_1,POLARLOCATION_2,RIMLOCATION_0,RIMLOCATION_1,RIMLOCATION_2,RENALSINUS_0,RENALSINUS_1,RENALSINUS_2,EXOPHYTICRATE_0,EXOPHYTICRATE_1,EXOPHYTICRATE_2,EXOPHYTICRATE_3,CLINICALSIZEGROUP_0,CLINICALSIZEGROUP_1,CLINICALSIZEGROUP_2,CLINICALSIZEGROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,CM_4,CM_5,RADIUSmaximaldiameterincm_0,RADIUSmaximaldiameterincm_1,RADIUSmaximaldiameterincm_2,RADIUSmaximaldiameterincm_3,EXOPHYTICENDOPHYTICPROPERTIES_0,EXOPHYTICENDOPHYTICPROPERTIES_1,EXOPHYTICENDOPHYTICPROPERTIES_2,EXOPHYTICENDOPHYTICPROPERTIES_3,EXOPHYTICENDOPHYTICPROPERTIES_4,NEARNESSOFTUMOUR_0,NEARNESSOFTUMOUR_1,NEARNESSOFTUMOUR_2,NEARNESSOFTUMOUR_3,ANTERIORORPOSTERIOR_0,ANTERIORORPOSTERIOR_1,ANTERIORORPOSTERIOR_2,ANTERIORORPOSTERIOR_3,LOCATIONTOPOLARLINE_0,LOCATIONTOPOLARLINE_1,LOCATIONTOPOLARLINE_2,LOCATIONTOPOLARLINE_3,PARTIALNEPHROINDICATION_0,PARTIALNEPHROINDICATION_1,PARTIALNEPHROINDICATION_2,PARTIALNEPHROINDICATION_3,MULTIFOCALITY_0,MULTIFOCALITY_1,MULTIFOCALITY_2
0,65.0,28.80,45.0,11.50,33.00,3000.00,1.00,55.64,1.00,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,30.0,24.09,40.0,16.20,46.50,5500.00,0.85,105.84,1.00,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,53.0,28.10,65.0,14.90,42.90,6700.00,0.72,84.73,1.00,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,76.0,36.20,31.0,13.91,41.56,7205.87,0.63,132.00,1.05,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,71.0,24.30,46.0,13.40,39.40,5820.00,0.89,84.26,1.00,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0

In [7]:
target = '30_day_COMPLICATIONS'
x_train = nn_train.drop(labels=target, axis = 'columns').copy()
y_train = nn_train[target].copy()
x_test = nn_test.drop(labels=target, axis = 'columns').copy()
y_test = nn_test[target].copy()

In [8]:
# Split train and validation set
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.20, random_state=42, stratify = y_train)

In [9]:
print("Number of records in trainset {}".format(len(x_train)))
print("Number of records in validset {}".format(len(x_valid)))
print("Number records in testset {}".format(len(x_test)))
print('% pos labels train {:.2f}'.format(y_train.sum()/len(y_train)))
print('% pos labels valid {:.2f}'.format(y_valid.sum()/len(y_valid)))
print('% pos labels test {:.2f}'.format(y_test.sum()/len(y_test)))

Number of records in trainset 1596
Number of records in validset 399
Number records in testset 856
% pos labels train 0.09
% pos labels valid 0.09
% pos labels test 0.09


In [10]:
# Standardize the numeric columns
from sklearn.preprocessing import StandardScaler
stdc = StandardScaler()
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

x_valid.reset_index(drop=True, inplace=True)
y_valid.reset_index(drop=True, inplace=True)

x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

float_col = train.select_dtypes('float').columns
x_train[float_col] = pd.DataFrame(stdc.fit_transform(x_train[float_col]),columns=float_col).copy()
x_valid[float_col] = pd.DataFrame(stdc.transform(x_valid[float_col]),columns=float_col).copy()
x_test[float_col] = pd.DataFrame(stdc.transform(x_test[float_col]),columns=float_col).copy()

In [11]:
x_train.head()

,AGEATSURGERY,BMI,CLINICALSIZEmm,PREOPHB,PREOPHT,PREOPWBC,PREOPCREAT,PREOPEGFR,NOOFLESIONS,GENDER_0,GENDER_1,GENDER_2,MARITALSTATUS_0,MARITALSTATUS_1,MARITALSTATUS_2,MARITALSTATUS_3,MARITALSTATUS_4,RACE_0,RACE_1,RACE_2,RACE_3,RACE_4,EDUCATION_0,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,ECOG_0,ECOG_1,ECOG_2,ECOG_3,ECOG_4,SYMPTOMS_0,SYMPTOMS_1,SYMPTOMS_2,SYMPTOMS_3,SOLITARYKIDNEY_0,SOLITARYKIDNEY_1,SOLITARYKIDNEY_2,SOLITARYKIDNEY_3,BILATERALITYOFTUMOR_0,BILATERALITYOFTUMOR_1,BILATERALITYOFTUMOR_2,SIDEOFTUMOR_0,SIDEOFTUMOR_1,SIDEOFTUMOR_2,SIDEOFTUMOR_3,SIDEOFSURGERY_0,SIDEOFSURGERY_1,SIDEOFSURGERY_2,SIDEOFSURGERY_3,FACE_0,FACE_1,FACE_2,TUMORlOCATION_0,TUMORlOCATION_1,TUMORlOCATION_2,TUMORlOCATION_3,TUMORlOCATION_4,TUMORlOCATION_5,POLARLOCATION_0,POLARLOCATION_1,POLARLOCATION_2,RIMLOCATION_0,RIMLOCATION_1,RIMLOCATION_2,RENALSINUS_0,RENALSINUS_1,RENALSINUS_2,EXOPHYTICRATE_0,EXOPHYTICRATE_1,EXOPHYTICRATE_2,EXOPHYTICRATE_3,CLINICALSIZEGROUP_0,CLINICALSIZEGROUP_1,CLINICALSIZEGROUP_2,CLINICALSIZEGROUP_3,CT_0,CT_1,CT_2,CT_3,CT_4,CT_5,CN_0,CN_1,CN_2,CN_3,CM_0,CM_1,CM_2,CM_3,CM_4,CM_5,RADIUSmaximaldiameterincm_0,RADIUSmaximaldiameterincm_1,RADIUSmaximaldiameterincm_2,RADIUSmaximaldiameterincm_3,EXOPHYTICENDOPHYTICPROPERTIES_0,EXOPHYTICENDOPHYTICPROPERTIES_1,EXOPHYTICENDOPHYTICPROPERTIES_2,EXOPHYTICENDOPHYTICPROPERTIES_3,EXOPHYTICENDOPHYTICPROPERTIES_4,NEARNESSOFTUMOUR_0,NEARNESSOFTUMOUR_1,NEARNESSOFTUMOUR_2,NEARNESSOFTUMOUR_3,ANTERIORORPOSTERIOR_0,ANTERIORORPOSTERIOR_1,ANTERIORORPOSTERIOR_2,ANTERIORORPOSTERIOR_3,LOCATIONTOPOLARLINE_0,LOCATIONTOPOLARLINE_1,LOCATIONTOPOLARLINE_2,LOCATIONTOPOLARLINE_3,PARTIALNEPHROINDICATION_0,PARTIALNEPHROINDICATION_1,PARTIALNEPHROINDICATION_2,PARTIALNEPHROINDICATION_3,MULTIFOCALITY_0,MULTIFOCALITY_1,MULTIFOCALITY_2
0,-0.841078,-0.726105,-1.215099,1.502499,0.686882,-1.916001,0.159917,0.000349,-0.207765,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.357387,1.100301,0.262373,-1.550309,-1.602868,1.026342,-1.544317,1.140371,-0.207765,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.635750,-0.654810,0.402733,-1.148623,-0.887321,-1.497535,-0.132238,0.033971,0.025095,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.996569,1.995680,-0.254742,-0.264916,-0.486615,0.830186,0.062532,-1.381873,9.106653,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.996569,-0.833047,-1.362846,-0.023905,-0.028665,-0.739064,0.452071,-1.711241,-0.207

In [12]:
from sklearn.utils.class_weight import compute_class_weight

In [13]:
weights = compute_class_weight(class_weight='balanced', classes=y_train.unique(), y = y_train)

In [14]:
weights

array([0.54920853, 5.58041958])

In [15]:
class_weight = {0: weights[0] , 1: weights[1]}

In [27]:
# input layer
np.random.seed(1)
tf.random.set_seed(1)
input_layer = keras.layers.Input(shape=(x_train.shape[1],), name = "input_layer")
x = keras.layers.Dense(128, name = 'Dense_1',activation='relu')(input_layer)
x = keras.layers.Dropout(0.4, name=  'Dropout_1', seed = 42)(x)
x = keras.layers.Dense(128, name = 'Dense_2',activation='relu')(x)
x = keras.layers.Dropout(0.4, name=  'Dropout_2', seed = 42)(x)
main_output = keras.layers.Dense(1, activation='sigmoid',name='main_output')(x)

model = keras.Model(inputs= input_layer, outputs=main_output)

# compiling the model
model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(curve = 'ROC',name = 'AUC_ROC'),
                       tf.keras.metrics.AUC(curve = 'PR', name = 'AUC_PR')],
              )

# Keras callback. The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min',restore_best_weights=True)


model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 121)]             0         
_________________________________________________________________
Dense_1 (Dense)              (None, 128)               15616     
_________________________________________________________________
Dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
Dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
Dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
main_output (Dense)          (None, 1)                 129       
Total params: 32,257
Trainable params: 32,257
Non-trainable params: 0
_______________________________________________________

In [28]:
model.fit(x = x_train, y = y_train,
          validation_data = (x_valid, y_valid),
          epochs = 100,
          callbacks = [early_stop],
          class_weight=class_weight
     )


Train on 1596 samples, validate on 399 samples
Epoch 1/100
1596/1596 [==============================] - 1s 792us/sample - loss: 0.6843 - AUC_ROC: 0.7027 - AUC_PR: 0.1661 - val_loss: 0.5223 - val_AUC_ROC: 0.7901 - val_AUC_PR: 0.2002
Epoch 2/100
1596/1596 [==============================] - 0s 92us/sample - loss: 0.5424 - AUC_ROC: 0.8034 - AUC_PR: 0.2638 - val_loss: 0.4922 - val_AUC_ROC: 0.7633 - val_AUC_PR: 0.1686
Epoch 3/100
1596/1596 [==============================] - 0s 89us/sample - loss: 0.5227 - AUC_ROC: 0.8250 - AUC_PR: 0.3909 - val_loss: 0.8146 - val_AUC_ROC: 0.7704 - val_AUC_PR: 0.2039
Epoch 4/100
1596/1596 [==============================] - 0s 86us/sample - loss: 0.4926 - AUC_ROC: 0.8403 - AUC_PR: 0.3481 - val_loss: 0.6431 - val_AUC_ROC: 0.7735 - val_AUC_PR: 0.2258
Epoch 5/100
1596/1596 [==============================] - 0s 86us/sample - loss: 0.4566 - AUC_ROC: 0.8692 - AUC_PR: 0.4997 - val_loss: 0.4125 - val_AUC_ROC: 0.7786 - val_AUC_PR: 0.2169
Epoch 6/100
1596/1596 [=========

In [29]:
# predict on test set
pd.DataFrame(model.predict(x_test))

,0
0,5.497567e-01
1,1.224279e-01
2,7.185674e-01
3,2.248251e-02
4,7.664124e-01
5,1.492264e-01
6,2.666175e-01
7,5.568232e-01
8,1.013279e-06
9,5.202913e-01


In [32]:
results_nn = pd.DataFrame(model.predict(x_test), columns=['pred_prob'])
results_nn['pred_label'] =  results_nn['pred_prob'].apply(lambda x: 1 if x>=0.5 else 0)
results_nn['true_label'] = np.array(y_test)

In [33]:
# NeuralNetwork Score Raw Data
print("\n Model Balanced Accuracy: \n" + str(metrics.balanced_accuracy_score(results_nn['true_label'], results_nn['pred_label'])))
print("\n Confusion Matrix : \n"+str(metrics.confusion_matrix(results_nn['true_label'], results_nn['pred_label'])))
print("\n Classification Report: \n"+ str(metrics.classification_report(results_nn['true_label'], results_nn['pred_label'])))
print("\n AUC-ROC: \n"+ str(metrics.roc_auc_score(results_nn['true_label'], results_nn['pred_prob'])))


def calc_aucpr_data(result):
    y_ACTUAL = result['true_label']
    scores_prob = result['pred_prob']
    yhat = result['pred_label']
    precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
    prc_auc = metrics.auc(recall,precision)
    return prc_auc

print("\n AUC-ROC: \n"+ str(calc_aucpr_data(results_nn)))


 Model Balanced Accuracy: 
0.6669389660403781

 Confusion Matrix : 
[[685  94]
 [ 42  35]]

 Classification Report: 
              precision    recall  f1-score   support

           0       0.94      0.88      0.91       779
           1       0.27      0.45      0.34        77

    accuracy                           0.84       856
   macro avg       0.61      0.67      0.62       856
weighted avg       0.88      0.84      0.86       856


 AUC-ROC: 
0.8090125535568411

 AUC-ROC: 
0.3716246639746523


In [58]:
from sklearn.utils import resample
resample()

In [59]:
def bootstrapped_AUC(result):
    from sklearn.utils import resample
    from tqdm import tqdm

    n_iter = 10000
    roc_auc = list()
    prc_auc = list()


    for i in range(n_iter):
        result_sample = resample(result, n_samples = len(result),random_state=i)
        
        #Calculating AUROC for each sample
        y_ACTUAL= result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        fpr, tpr, thresholds = metrics.roc_curve(y_ACTUAL, scores_prob, pos_label=1)
        roc_auc.append(metrics.auc(fpr, tpr))

        #calculate AUPRC for each sample
        y_ACTUAL = result_sample['true_label']
        scores_prob = result_sample['pred_prob']
        yhat = result_sample['pred_label']
        precision, recall, thresholds = metrics.precision_recall_curve(y_ACTUAL, scores_prob, pos_label=1)
        prc_auc.append(metrics.auc(recall,precision))
    
    return roc_auc, prc_auc

In [60]:
roc_auc_nn, pr_auc_nn = bootstrapped_AUC(results_nn)

In [61]:
dict = {'roc_auc_nn': roc_auc_nn,
        'pr_auc_nn': pr_auc_nn,
       }
pd.DataFrame(dict).describe(percentiles=[0.025,0.975])

,roc_auc_nn,pr_auc_nn
count,10000.000000,10000.000000
mean,0.810780,0.328811
std,0.022705,0.052438
min,0.718993,0.118066
2.5%,0.764172,0.226977
50%,0.811511,0.328757
97.5%,0.853611,0.432983
max,0.896569,0.528865


In [62]:
output_path = 'output/models'

In [63]:
import os
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [64]:
model.save(filepath=output_path+'/nn_model.h5')

In [65]:
# Export Standcaler used to process the testset
from joblib import dump
dump(stdc, output_path+'/nn_stdc.joblib')

['output/models/nn_stdc.joblib']

In [66]:
# Export Standcaler used to process the testset
from joblib import dump
dump(encoder, output_path+'/nn_OHE.joblib')

['output/models/nn_OHE.joblib']